# Data With Danny
## 8 Week SQL Challenge (Week 1 Danny's Diner)

### <u>Introduction </u>

Danny's passion for Japanese cuisine led him to venture into the restaurant industry at the start of 2021. With a love for sushi, curry, and ramen, he opened a charming eatery named Danny's Diner. Now, Danny's Diner seeks your expertise to navigate the challenges of sustaining its operations. While the restaurant has collected fundamental data during its initial months, they lack the know-how to leverage this information effectively in managing their business.

<div>
<img src="./dannys_diner/dannys_diner_intro.png" width="500"/>
</div>

### <u>Problem Statement </u>
Danny aims to utilize the data to gain insights into his customers, focusing on their:
- Visiting patterns,
- Expenditure,
- Favorite menu items.

By understanding these aspects, Danny seeks to establish a deeper connection with his patrons, enhancing their dining experience with personalized touches. He intends to employ these findings in deciding whether to enhance the current customer loyalty program. Moreover, he requires assistance in creating basic datasets for his team to analyze the data seamlessly, eliminating the need for SQL.

The dataset comprises three tables: 
- sales
- members 
- menu 

Please refer to the relationship diagram below for understanding their connections.

### <u>Entity-Relationship Diagram</u>
<div>
<img src="./dannys_diner/entity-diagram.png" width="500"/>
</div>

##### 1. What is the total amount each customer spent at the restaurant?
```sql 
SELECT s.customer_id, SUM(price) AS total_spent FROM dannys_diner.sales AS s
LEFT JOIN dannys_diner.members AS m
ON s.customer_id = m.customer_id
LEFT JOIN dannys_diner.menu AS me
ON s.product_id = me.product_id
GROUP BY s.customer_id 
ORDER BY total_spent DESC

SELECT * FROM table1
```

<div>
<img src="./dannys_diner/s1.png" width="500"/>
</div>

##### 2. How many days has each customer visited the restaurant?


```sql
SELECT
  customer_id,
  COUNT(DISTINCT order_date)
FROM dannys_diner.sales
GROUP BY 1
ORDER BY 2 DESC
```

![image info](./dannys_diner/s2.png)

##### 3. What was the first item from the menu purchased by each customer?


```sql
WITH ranking_table AS (
SELECT m.customer_id, 
       s.order_date,
       RANK() OVER (PARTITION BY m.customer_id ORDER BY s.order_date) AS product_ranking,
       me.product_name
       FROM 
dannys_diner.members AS m 
JOIN dannys_diner.sales AS s
 ON m.customer_id = s.customer_id
JOIN dannys_diner.menu AS me 
 ON s.product_id = me.product_id
)

SELECT * FROM ranking_table
WHERE product_ranking = 1
```

![image info](./dannys_diner/s3.png)

##### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

```sql 
DROP TABLE IF EXISTS t1; 

CREATE TEMP TABLE t1 AS (
SELECT me.product_name, s.order_date FROM dannys_diner.sales AS s
JOIN dannys_diner.menu AS me 
ON s.product_id = me.product_id
);

SELECT product_name, COUNT(product_name) AS product_count 
FROM t1 
GROUP BY product_name
ORDER BY product_count DESC
```

![image info](./dannys_diner/s4a.png)

##### 5. Which item was the most popular for each customer?


```sql
DROP TABLE IF EXISTS t1; 

CREATE TEMP TABLE t1 AS (
SELECT s.customer_id, me.product_name, COUNT(me.product_name) AS product_count,
RANK() OVER (PARTITION BY s.customer_id ORDER BY COUNT(me.product_name) DESC) AS product_rank
FROM dannys_diner.sales AS s
JOIN dannys_diner.menu AS me 
ON s.product_id = me.product_id
JOIN dannys_diner.members AS m 
ON m.customer_id = s.customer_id
GROUP BY s.customer_id, me.product_name
);

SELECT customer_id, product_name, product_count
FROM t1 
WHERE product_rank = 1
```


![image info](./dannys_diner/s5.png)

##### 6. Which item was purchased first by the customer after they became a member?


```sql 
DROP TABLE IF EXISTS customer_buy;
CREATE TEMP TABLE customer_buy AS 
(
SELECT m.customer_id,
       m.join_date,
       me.product_name,
       s.order_date, 
       RANK() OVER (PARTITION BY m.customer_id ORDER BY s.order_date) AS order_rank
FROM dannys_diner.members AS m 
JOIN dannys_diner.sales AS s 
ON s.customer_id = m.customer_id 
JOIN dannys_diner.menu AS me 
ON me.product_id = s.product_id
WHERE s.order_date > m.join_date
);

SELECT * FROM customer_buy
WHERE order_rank = 1
```

![image info](./dannys_diner/s6.png)

##### 7. Which item was purchased just before the customer became a member?


```sql 
DROP TABLE IF EXISTS customer_buy;
CREATE TEMP TABLE customer_buy AS 
(
SELECT m.customer_id,
       me.product_name,
       DENSE_RANK() OVER (PARTITION BY m.customer_id ORDER BY MIN(s.order_date - m.join_date) DESC) AS product_rank 
FROM dannys_diner.members AS m 
JOIN dannys_diner.sales AS s 
ON s.customer_id = m.customer_id 
JOIN dannys_diner.menu AS me 
ON me.product_id = s.product_id
WHERE (s.order_date - m.join_date) < 0 
GROUP BY m.customer_id, me.product_name
);

SELECT *
FROM customer_buy
WHERE product_rank = 1
```


![image info](./dannys_diner/s7.png)

##### 8. What is the total items and amount spent for each member before they became a member?

```sql 
DROP TABLE IF EXISTS customer_buy; 

CREATE TEMP TABLE customer_buy AS (
SELECT m.customer_id,
       m.join_date, 
       s.order_date, 
       me.product_name,
       me.price
FROM dannys_diner.members AS m 
JOIN dannys_diner.sales AS s 
ON m.customer_id = s.customer_id 
JOIN dannys_diner.menu AS me 
ON me.product_id = s.product_id
WHERE s.order_date < m.join_date
);

WITH t2 AS (
  SELECT customer_id, product_name, COUNT(DISTINCT product_name) AS unique_products, SUM(price) AS total_spent FROM customer_buy
  GROUP BY customer_id, product_name
)

SELECT customer_id, SUM(unique_products) AS total_products, SUM(total_spent) 
FROM t2 
GROUP BY customer_id
```

![image info](./dannys_diner/s8.png)

##### 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

```sql 
DROP TABLE IF EXISTS customer_points; 

CREATE TEMP TABLE customer_points AS (
SELECT s.customer_id, me.product_name, me.price 
FROM dannys_diner.members AS m 
RIGHT JOIN dannys_diner.sales AS s 
ON m.customer_id = s.customer_id 
JOIN dannys_diner.menu AS me 
ON me.product_id = s.product_id
);

WITH points_table AS (
SELECT customer_id, product_name,
CASE 
  WHEN product_name = 'sushi' 
    THEN (price * 2 * 10) 
    ELSE (price * 10) 
  END AS points
FROM customer_points
)

SELECT customer_id, SUM(points) AS total_points FROM points_table 
GROUP BY customer_id 
ORDER BY SUM(points)

```

![image info](./dannys_diner/s9.png)

##### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

```sql 
SELECT
  sales.customer_id,
  SUM(
    CASE
      WHEN sales.order_date BETWEEN
        members.join_date::DATE AND members.join_date::DATE+6
        THEN 2 * 10 * menu.price
      WHEN menu.product_name = 'sushi' THEN 2 * 10 * menu.price 
      ELSE 10 * menu.price
  END) AS points
FROM dannys_diner.sales
JOIN dannys_diner.menu
  ON sales.product_id = menu.product_id
JOIN dannys_diner.members
  ON sales.customer_id = members.customer_id
WHERE sales.order_date <= CAST('2021-01-31' AS DATE)
GROUP BY sales.customer_id
ORDER BY customer_id
```

![image info](./dannys_diner/s10.png)

#### Bonus questions
##### 11. Recreate the following table output using the available data:

![image info](./dannys_diner/s11.png)

```sql
SELECT s.customer_id, 
       s.order_date, 
       me.product_name, 
       me.price, 
       CASE WHEN m.join_date <= s.order_date THEN 'Y' ELSE 'N' END AS member
FROM dannys_diner.sales AS s 
LEFT JOIN dannys_diner.menu AS me 
ON s.product_id = me.product_id
LEFT JOIN dannys_diner.members AS m 
ON m.customer_id = s.customer_id 
ORDER BY customer_id, order_date, product_name
```

##### 12. Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

![image info](./dannys_diner/s12.png)

```sql
WITH t1 AS (
SELECT s.customer_id, 
       s.order_date, 
       me.product_name, 
       me.price, 
       CASE WHEN m.join_date <= s.order_date THEN 'Y' ELSE 'N' END AS member
FROM dannys_diner.sales AS s 
LEFT JOIN dannys_diner.menu AS me 
ON s.product_id = me.product_id
LEFT JOIN dannys_diner.members AS m 
ON m.customer_id = s.customer_id 
)

SELECT *, 
CASE WHEN member = 'Y' THEN RANK() OVER 
(PARTITION BY customer_id, member ORDER BY order_date) 
ELSE NULL
END AS ranking
FROM t1
```